<a href="https://colab.research.google.com/github/abhinav9629/Sentiment-Classifier/blob/main/Sentiment_Analysis_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [78]:
import tensorflow.keras as keras
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [98]:
train_df = pd.read_csv('/content/MyDrive/MyDrive/HOTEL REVIEW DATASET/SAD/train.csv',encoding='cp1252')
test_df = pd.read_csv('/content/MyDrive/MyDrive/HOTEL REVIEW DATASET/SAD/test.csv',encoding='cp1252')


In [99]:
train_df = train_df[['text','sentiment']]
test_df = test_df[['text','sentiment']]


In [100]:
train_df['text'] = train_df['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))
train_df['text'] = train_df['text'].apply(lambda x : str(x).lower())
test_df['text'] = test_df['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))
test_df['text'] = test_df['text'].apply(lambda x : str(x).lower())


In [101]:
train_targets = pd.get_dummies(train_df['sentiment']).values

In [102]:
test_targets = pd.get_dummies(test_df['sentiment']).values

In [84]:
for idx,row in train_df.iterrows():
    row[0] = row[0].replace('rt',' ')

for idx,row in test_df.iterrows():
    row[0] = row[0].replace('rt',' ')

In [117]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_df['text'].values)
train_samples = tokenizer.texts_to_sequences(train_df['text'].values)
train_samples = pad_sequences(train_samples)


In [86]:
#Hyperparameters
embed_dim = 128
lstm_out = 196

In [62]:
#MODEL

In [87]:
model = keras.Sequential([
  keras.layers.Embedding(max_features,embed_dim,input_length=train_samples.shape[1]),
  keras.layers.SpatialDropout1D(0.4),
  keras.layers.LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2),
  keras.layers.Dense(3,activation='softmax')                          
])


In [88]:
optimiser = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,loss="categorical_crossentropy",metrics=['accuracy'])

In [89]:
train_samples.shape

(27481, 33)

In [90]:
train_targets.shape

(27481, 3)

In [92]:
model.fit(train_samples,train_targets,batch_size=32,epochs=50)

Epoch 1/50
859/859 [==============================] - 156s 182ms/step - loss: 0.7199 - accuracy: 0.6965
Epoch 2/50
859/859 [==============================] - 154s 180ms/step - loss: 0.6412 - accuracy: 0.7372
Epoch 3/50
859/859 [==============================] - 155s 180ms/step - loss: 0.6110 - accuracy: 0.7498
Epoch 4/50
859/859 [==============================] - 155s 180ms/step - loss: 0.5851 - accuracy: 0.7623
Epoch 5/50
859/859 [==============================] - 155s 181ms/step - loss: 0.5591 - accuracy: 0.7704
Epoch 6/50
859/859 [==============================] - 154s 179ms/step - loss: 0.5404 - accuracy: 0.7788
Epoch 7/50
859/859 [==============================] - 154s 180ms/step - loss: 0.5191 - accuracy: 0.7895
Epoch 8/50
859/859 [==============================] - 154s 179ms/step - loss: 0.4948 - accuracy: 0.7997
Epoch 9/50
859/859 [==============================] - 155s 180ms/step - loss: 0.4742 - accuracy: 0.8068
Epoch 10/50
859/859 [==============================] - 154s 179m

In [115]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(test_df['text'].values)
test_samples = tokenizer.texts_to_sequences(test_df['text'].values)
test_samples = pad_sequences(test_samples)

In [116]:
model.evaluate(test_samples,test_targets)

151/151 [==============================] - 3s 22ms/step - loss: 3.2586 - accuracy: 0.2629


[3.258634090423584, 0.2629283368587494]